In [122]:
import os

import requests
import urllib3
urllib3.disable_warnings()

from bs4 import BeautifulSoup

In [105]:
dates = []
urls = []

for i in range(0, 28):
    base_url = f'https://estadisticas.pr/index.php/en/estadisticas-mas-recientes'
    
    page = requests.get(base_url, 
                        params = {'type': 'precios_mayoristas_gasolina',
                                 'page': str(i)},
                        verify = False)
    bs = BeautifulSoup(page.content, 'html')
    
    entries = bs.find('div', {'class': 'view-content'}).find('tbody').findAll('tr')
    
    for entry in entries:
        try:
            file_type = entry.find('td', {'class': 'views-field-field-estadisticas'}).find('span')['class']

            if 'file--application-pdf' in file_type:
                date = entry.find('td', {'class': 'views-field-field-fecha-de-publicacion'}).text
                date = date.strip()
                dates.append(date)
                
                url = entry.find('a')['href']
                urls.append(url)
        except:
            continue
            
assert len(urls) == len(dates), print('Dates and URLS don\'t match')

In [111]:
if 'pdf_files' not in os.listdir('.'):
    os.makedirs('pdf_files')
else:
    os.rmdir('pdf_files')
    os.makedirs('pdf_files')

In [112]:
def get_file(date, url, file_path = './'):
    file = requests.get(url, verify = False)
    
    # From: https://www.geeksforgeeks.org/downloading-files-web-using-python/
    with open(f'{file_path}{date}.pdf', 'wb') as pdf:
        for chunk in file.iter_content(chunk_size = 1024):
            if chunk:
                pdf.write(chunk)
                
    return None

In [121]:
for i in range(0, len(dates)):
    get_file(dates[i], urls[i], file_path = 'pdf_files/')